In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import random
from random import randrange
from datetime import timedelta

In [ ]:
environmental_history = pd.read_csv('data\environmental_data_history_br.csv')
companies = pd.read_csv('data\companies_br.csv')
companies_financials = pd.read_csv('data\companies_financials_br.csv')
companies_shares = pd.read_excel('data\companies_total_outstading_shares.xlsx', sheet_name = "Sheet1")

In [ ]:
ids = environmental_history['company_id'].unique()
companies = companies[companies['company_id'].isin(ids)]
companies = companies[['company_id', 'industry', 'company_name', 'ticker']]
companies.reset_index(inplace = True, drop = True)
companies.head(10)

In [ ]:
companies_shares = companies_shares[['Ticker_Curto', 'LTM2021', 'FY2020', 'FY2019',
       'FY2018', 'FY2017', 'FY2016', 'FY2015', 'FY2014', 'FY2013']]
companies_shares.rename(columns = {"LTM2021" : "2021", "FY2020": "2020", "FY2019" : "2019", 
                                   "FY2018" : "2018", "FY2017" : "2017", "FY2016" : "2016", 
                                   "FY2015" : "2015", "FY2014" : "2014", "FY2013" : "2013",
                                   "Ticker_Curto" : "ticker"}, inplace = True)
companies_shares.reset_index(inplace = True, drop = True)
companies_shares = companies_shares.melt(id_vars=["ticker"], var_name="fiscal_year", value_name='out_shares')
companies_shares.dropna(inplace = True)
companies_shares['fiscal_year'] = companies_shares['fiscal_year'].apply(lambda x: int(x))
companies_shares.head()

In [ ]:
# unit -> Millions BRL
important = ['EBITDA', 'EBT, Excl. Unusual Items', 'Total Revenues']
companies_financials = companies_financials.loc[companies_financials['company_id'].isin(ids)]
companies_financials = companies_financials.loc[companies_financials['data_item'].isin(important)]
companies_financials = companies_financials[['company_id', 'ref_date', 'data_item_value', 'data_item']]
companies_financials['ref_date'] = companies_financials['ref_date'].apply(lambda x: int(x.replace(x, x.split('-')[0])))
companies_financials = companies_financials.pivot_table(index = ['company_id', 'ref_date'], columns='data_item', values='data_item_value').reset_index()
companies_financials.rename(columns = {"ref_date" : "fiscal_year", "EBT, Excl. Unusual Items" : "EBT", 'Total Revenues' : 'total_revenues'}, inplace = True)

In [ ]:
companies_informations = pd.merge(companies_financials, companies, on = ['company_id'])
companies_informations = pd.merge(companies_informations, companies_shares, on = ['ticker', 'fiscal_year'])
companies_informations.drop('company_id', axis = 1, inplace = True)
companies_informations.head()

In [ ]:
# unit -> tCO2e
scopes = ['Absolute: Greenhouse Gases Scope 1', 'Absolute: Greenhouse Gases Scope 2']
environmental_history = environmental_history.loc[environmental_history['data_item_name'].isin(scopes)]
environmental_history = environmental_history[['company_id', 'fiscal_year', 'data_item_name', 'data_item_value']]
environmental_history = environmental_history.groupby(['company_id', 'fiscal_year'], as_index = False)['data_item_value'].sum()
environmental_history.rename(columns = {"data_item_value" : "emissions"}, inplace = True)
environmental_history.reset_index(inplace = True, drop = True)
environmental_history.head()

In [ ]:
environmental_history = environmental_history.join(companies.set_index('company_id'), on = 'company_id')
environmental_history = environmental_history.loc[environmental_history['fiscal_year'].ge(2013)]
environmental_history.drop(['company_id', 'company_name', 'industry'], axis = 1, inplace = True)
environmental_history.reset_index(inplace = True, drop = True)
environmental_history.head()

In [ ]:
companies_history = environmental_history.merge(companies_informations, on = ['ticker', 'fiscal_year'])
companies_history.head()

In [ ]:
tickers = companies_history['ticker'].unique()
average_price = {}
last_price = {}
for ticker in tickers:
    try:
        years = np.sort(companies_history.loc[companies_history['ticker'].eq(ticker)]['fiscal_year'].to_list())
        data = yf.download(ticker + '.SA', start = datetime(years[0], 1, 1), end = datetime(years[-1] + 1, 1, 1))
        data.reset_index(inplace = True)
        data['year'] = pd.DatetimeIndex(data['Date']).year
        tmp = {}
        for year in data['year'].unique():
            tmp[year] = data.loc[data['year'].eq(year)].iloc[-1]['Close']
        last_price[ticker] = pd.Series(tmp)
        average_price[ticker] = data.groupby('year')['Close'].mean()
    except:
        print(ticker)

In [ ]:
def transform_dict_into_dataframe(dic, value_column):
    df = pd.DataFrame.from_dict(dic, orient = 'index')
    df.reset_index(inplace = True)
    df.rename(columns = {"index" : "ticker"}, inplace = True)
    df = df.melt(id_vars = ["ticker"], var_name = "fiscal_year", value_name = value_column)
    df.dropna(inplace = True)
    return df

In [ ]:
average_price = transform_dict_into_dataframe(average_price, 'average_price')
last_price = transform_dict_into_dataframe(last_price, 'last_price')

In [ ]:
price_data = average_price.merge(last_price, on = ['ticker', 'fiscal_year'])

In [ ]:
companies_history = companies_history.merge(price_data, on = ['ticker', 'fiscal_year'])
companies_history.to_csv('companies_history.csv')
companies_history.to_excel('companies_history_excel.xlsx')